In [1]:
%run supervised_functions.ipynb

# Generate data

In [2]:
start_mols = pickle.load(open("datasets/my_uspto/unique_start_mols.pickle", 'rb'))

In [3]:
np.random.seed(42)

N = 100000
steps = 1

df_list = []
final_shape = 0
smiles_per_random_sample = 1000
pool_chunk_size = 10

# Create dataset for multi-step pred
with Pool(30) as p, tqdm.tqdm(total=N) as pbar:
    while final_shape < N:
        smiles = np.random.choice(start_mols, size=(smiles_per_random_sample,))

        for new_df in p.imap_unordered(functools.partial(generate_train_data, steps=steps), smiles, chunksize=10):
            df_list.append(new_df)
            final_shape += new_df.shape[0]
            
        pbar.update(final_shape - pbar.n)

main_df = pd.concat(df_list)
del df_list
print(main_df.shape)

# randomize
main_df = pd.concat([main_df[:int(main_df.shape[0]*0.8)].sample(frac=1), main_df[int(main_df.shape[0]*0.8):].sample(frac=1)])
print(main_df.shape)

100272it [04:03, 411.06it/s]                                                                         


(100272, 10)
(100272, 10)


# Networks

In [4]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

# Helper stuff

In [5]:
%%time
action_dataset = pd.read_csv("datasets/my_uspto/action_dataset-filtered.csv", index_col=0)
action_dataset = action_dataset.loc[action_dataset["action_tested"] & action_dataset["action_works"]]
action_dataset = action_dataset[["rsub", "rcen", "rsig", "rbond", "psub", "pcen", "psig", "pbond"]]
print(action_dataset.shape)

action_rsigs = data.Molecule.pack(list(map(molecule_from_smile, action_dataset["rsig"])))
action_psigs = data.Molecule.pack(list(map(molecule_from_smile, action_dataset["psig"])))

(89384, 8)


/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `Li`
  warnings.warn("Unknown value `%s`" % x)
/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `Ge`
  warnings.warn("Unknown value `%s`" % x)
/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `K`
  warnings.warn("Unknown value `%s`" % x)
/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `Na`
  warnings.warn("Unknown value `%s`" % x)
/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `Ti`
  warnings.warn("Unknown value `%s`" % x)
/home/abhor/miniconda3/envs/de_

CPU times: user 4min 56s, sys: 17 s, total: 5min 13s
Wall time: 4min 58s


# Training

In [6]:
actor_lr = 3e-4
epochs = 50
batch_size = 512

#####################
# Get actor network #
#####################
actor = PolicyNetwork().to(device)#
actor.eval() # Non-trainable for now
pass

In [ ]:
################################################################
# Get inital embeddings and applicable_indices+correct_indices #
################################################################
action_embeddings = get_action_dataset_embeddings(actor.GIN)
print(action_embeddings.shape)

# I'm storing as lists, so doing numpy operations for the elements
correct_applicable_indices = []
correct_action_dataset_indices = []
action_embedding_indices = []

# for indices_used_for_data, correct_idx in tqdm.tqdm(map(get_emb_indices_and_correct_idx, main_df.iterrows()), total=main_df.shape[0]):
with Pool(20) as p:
    for indices_used_for_data, correct_app_idx, correct_act_idx in tqdm.tqdm(p.imap(get_emb_indices_and_correct_idx, main_df.iterrows(), chunksize=50), total=main_df.shape[0]):
        action_embedding_indices.append(indices_used_for_data)
        correct_applicable_indices.append(correct_app_idx)
        correct_action_dataset_indices.append(correct_act_idx)

100%|████████████████████████████████████████████████████████████████| 44/44 [00:09<00:00,  4.82it/s]

torch.Size([89384, 256])



  2%|█                                                        | 1951/100272 [00:05<02:23, 685.99it/s]

In [ ]:
# Make training data
def make_data(args):
    i, negative_topk = args
    data = []
    targets = []
    act_emb_for_i, correct_index = action_embeddings[action_embedding_indices[i]], correct_indices[i]

    # Positive
    positive = act_emb_for_i[correct_index]

    # Calc negatives
    dist = np.linalg.norm(action_embeddings - positive, axis=1)
    sorted_idx = np.argsort(dist)[:negative_topk] 
    sorted_idx = sorted_idx[sorted_idx != correct_index] # Remove correct index in list
    
    data = np.array([(i, correct_index)]+list(zip([i] * len(sorted_idx), sorted_idx)))
    targets = [[1] + [0] * len(sorted_idx)]
    return data, targets

train_X = []
train_Y = []
test_X = []
test_Y = []
with Pool(20) as p:
    bs = 100
    # Train
    start, end = 0, int(main_df.shape[0]*0.8)
    for X, Y in tqdm.tqdm(
                          p.imap_unordered(make_data, 
                                 [(i, 10) for i in range(start, end)], 
                                 chunksize=bs), 
                          total=(end-start)):
        train_X.append(X)
        train_Y.append(Y)
    
    # Test
    start, end = int(main_df.shape[0]*0.8), main_df.shape[0]
    for X, Y in tqdm.tqdm(
                          p.imap_unordered(make_data, 
                                 [(i, 1) for i in range(start, end)], 
                                 chunksize=bs), 
                          total=end-start):
        test_X.append(X)
        test_Y.append(Y)


train_X = np.concatenate(train_X, axis=0)
test_X = np.concatenate(test_X, axis=0)

In [ ]:
%time train_reactants = data.Molecule.pack(list(map(molecule_from_smile, main_df.iloc[train_X[:, 0]]["reactant"])))
%time train_products = data.Molecule.pack(list(map(molecule_from_smile, main_df.iloc[train_X[:, 0]]["product"])))
%time train_rsigs = data.Molecule.pack(list(map(molecule_from_smile, action_dataset.iloc[train_X[:, 1]]["rsig"])))
%time train_psigs = data.Molecule.pack(list(map(molecule_from_smile, action_dataset.iloc[train_X[:, 1]]["psig"])))

test_reactants = data.Molecule.pack(list(map(molecule_from_smile, main_df.iloc[test_X[:, 0]]["reactant"])))
test_products = data.Molecule.pack(list(map(molecule_from_smile, main_df.iloc[test_X[:, 0]]["product"])))
test_rsigs = data.Molecule.pack(list(map(molecule_from_smile, action_dataset.iloc[test_X[:, 1]]["rsig"])))
test_psigs = data.Molecule.pack(list(map(molecule_from_smile, action_dataset.iloc[test_X[:, 1]]["psig"])))

train_Y = torch.Tensor(np.concatenate(train_Y, axis=1)).view(-1, 1)
test_Y = torch.Tensor(np.concatenate(test_Y, axis=1)).view(-1, 1)

print(train_reactants.batch_size, train_products.batch_size, train_rsigs.batch_size, train_psigs.batch_size, train_Y.shape)

print(test_reactants.batch_size, test_products.batch_size, test_rsigs.batch_size, test_psigs.batch_size, test_Y.shape)

In [ ]:
torch.cuda.empty_cache()    

In [ ]:
##########
# Critic #
##########
critic_lr = 1e-3
epochs = 5
num_hidden = 2
hidden_size = 256



torch.cuda.empty_cache()    
critic = CriticNetwork().to(device)
print(critic.DENSE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(critic.parameters(), lr=critic_lr)  

loss_list = []
test_loss = []

# Train the critic
num_batch_per_gpu_transfer = 100
for epoch in range(epochs):
    critic.train()
    for i in tqdm.tqdm(range(0, train_reactants.batch_size - batch_size, batch_size)):
        if i % (batch_size * num_batch_per_gpu_transfer) == 0:
            batch_reactants = train_reactants[i:min(i+batch_size*num_batch_per_gpu_transfer, train_reactants.batch_size)].to(device)
            batch_products = train_products[i:min(i+batch_size*num_batch_per_gpu_transfer, train_reactants.batch_size)].to(device)
            batch_rsigs = train_rsigs[i:min(i+batch_size*num_batch_per_gpu_transfer, train_reactants.batch_size)].to(device)
            batch_psigs = train_psigs[i:min(i+batch_size*num_batch_per_gpu_transfer, train_reactants.batch_size)].to(device)
            batch_Y = train_Y[i:min(i+batch_size*num_batch_per_gpu_transfer, train_reactants.batch_size)].to(device)

        # Forward pass
        j = i % batch_size*num_batch_per_gpu_transfer
        outputs = critic(batch_reactants[j:j+batch_size], batch_products[j:j+batch_size], batch_rsigs[j:j+batch_size], batch_psigs[j:j+batch_size])
        loss = criterion(outputs, batch_Y[j:j+batch_size].to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Free memory
    torch.cuda.empty_cache()    
    loss_list.append(loss.item())

    # Get test loss
    critic.eval()
    temp_test_loss = []
    for i in range(0, test_reactants.batch_size - batch_size, batch_size):
        temp_test_loss.append(
                        criterion(critic(test_reactants[i:i+batch_size].to(device), 
                                         test_products[i:i+batch_size].to(device), 
                                         test_rsigs[i:i+batch_size].to(device), 
                                         test_psigs[i:i+batch_size].to(device)) > 0.5, 
                                  test_Y[i:i+batch_size].to(device)).item()) 
        torch.cuda.empty_cache()    
    test_loss.append(np.mean(temp_test_loss))
    
    # Free memory
    torch.cuda.empty_cache()   

    print ('Epoch {}, Loss: {:.4f}, Test Loss: {:.4f}'.format(epoch+1, loss_list[-1], test_loss[-1]))

print("\nFINAL TEST LOSS:", test_loss[-1])

plt.plot(loss_list[5:], label="training loss")
plt.plot(test_loss[5:], label="test loss")
plt.legend()
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()



In [ ]:
torch.save(critic, f"models/supervised/critic/{steps}step.pth")

In [ ]:
list(zip(critic(test_reactants, test_products, test_rsigs, test_psigs), train_Y))